In [1]:
import influxdb_client
import os
import time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
# export your token into the environment variable INFLUXDB_TOKEN first
url = "http://10.10.10.247:8086"
# replace url with your server address
# if your server is on the same machine, use "http://localhost:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token)

In [4]:
bucket = "get-started"
write_api = write_client.write_api(write_options=SYNCHRONOUS)

for value in range(5):
    point = (
        Point("measurement1")
        .tag("tagname1", "tagvalue1")
        .field("field", value)
    )
    write_api.write(bucket=bucket, org='docs', record=point)
    # the default org is 'docs' if you deploy by docker-compose
    time.sleep(1)

In [7]:
query_api = write_client.query_api()
query = """from(bucket: "get-started")
|> range(start: -10m)
|> filter(fn: (r) => r._measurement == "measurement1")
|> mean() """
# Flux language
# Flux is an open source functional data scripting language 
# designed for querying, analyzing, and acting on data.
# for more details, please refer to flux language tutorial: https://docs.influxdata.com/flux/v0/

# retrieve the last 10 miniutes data and filter the measurement
# then calculate the mean value

tables = query_api.query(query, org="docs")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 1, 7, 12, 7, 46, 518795, tzinfo=tzlocal()), '_stop': datetime.datetime(2025, 1, 7, 13, 7, 46, 518795, tzinfo=tzlocal()), '_value': 2.0, '_field': 'field', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
